In [125]:
# To compare the performance of least squares, ridge regression and LASSO.
# Ideally, we should get accuracy increase in the same order.
from sklearn.datasets import load_boston
import numpy as np
from sklearn import preprocessing as skp
from sklearn import model_selection as sms
boston = load_boston()
X_orig = boston.data
y = boston.target

In [126]:
# The scaling and shifting need to be done for both the training and testing set but the information on how to 
# scale and shift must only be gleaned from the training set.
print(X_orig.shape)
print(y.shape)

(506, 13)
(506,)


In [127]:
# This is for standardizing by subtracting mean and unit variance...
# Do we have to decide the standardization method? 

X = X_orig
# X = skp.scale(X_orig)
# print(X.shape)

In [128]:
X_train, X_test, y_train, y_test = sms.train_test_split(X, y, test_size=0.208, random_state=76)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
mean_vec = np.mean(X_train, axis = 0)
std_vec = np.std(X_train, axis = 0)

mean_vecy = np.mean(y_train, axis = 0)
std_vecy = np.std(y_train, axis = 0)

# Need to define holdout sets for defining hyperparameters in the case of ridge regression and for LASSO. 
X_train_rr_lasso, X_val, y_train_rr_lasso, y_val = sms.train_test_split(X_train, y_train, test_size=0.25, random_state=42)

print(X_train_rr_lasso.shape)
print(X_val.shape)
print(y_train_rr_lasso.shape)
print(y_val.shape)

X_train = (X_train - mean_vec)/(std_vec)
X_test = (X_test - mean_vec)/(std_vec)
X_train_rr_lasso = (X_train_rr_lasso - mean_vec)/(std_vec)
X_val = (X_val - mean_vec)/(std_vec)
X_train = np.column_stack((X_train,np.ones(len(X_train))))
X_test = np.column_stack((X_test,np.ones(len(X_test))))
X_val = np.column_stack((X_val,np.ones(len(X_val))))
X_train_rr_lasso = np.column_stack((X_train_rr_lasso,np.ones(len(X_train_rr_lasso))))


y_train = (y_train - mean_vecy)#/(std_vecy)
y_test = (y_test - mean_vecy)#/(std_vecy)
y_train_rr_lasso = (y_train_rr_lasso - mean_vecy)#/(std_vecy)
y_val = (y_val - mean_vecy)#/(std_vecy)


(400, 13)
(106, 13)
(400,)
(106,)
(300, 13)
(100, 13)
(300,)
(100,)


In [129]:
theta = np.dot(np.dot((np.linalg.inv(np.dot(np.transpose(X_train),X_train))),np.transpose(X_train)),y_train)
y_pred = np.dot(X_test, theta)
MSE = ((np.linalg.norm(y_test - y_pred))**2)/len(y_pred)
print("MSE for normal linear regression: ", MSE)

MSE for normal linear regression:  16.34860534


In [130]:
# Ridge regression (lambda hyper parameter)
print("Validation scores are as follows: ")
lamset = np.array([0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 1000000])
for k in range(0,12):
    theta = np.dot(np.dot((np.linalg.inv(np.dot(np.transpose(X_train_rr_lasso),X_train_rr_lasso) + lamset[k]*np.identity(14))),np.transpose(X_train_rr_lasso)),y_train_rr_lasso)
    y_pred = np.dot(X_val, theta)
    MSE = ((np.linalg.norm(y_val - y_pred))**2)/len(y_pred)
    print("MSE for linear regression using lambda = ", lamset[k], "is: ",  MSE)
print("The best lambda: ", 100)
print("Ridge egression model trained on the entire set: ")
theta = np.dot(np.dot((np.linalg.inv(np.dot(np.transpose(X_train),X_train) + 100*np.identity(14))),np.transpose(X_train)),y_train)
y_pred = np.dot(X_test, theta)
MSE = ((np.linalg.norm(y_test - y_pred))**2)/len(y_pred)
print("MSE Score using best lambda = ",100, ': ', MSE)


Validation scores are as follows: 
MSE for linear regression using lambda =  1e-06 is:  22.8850062702
MSE for linear regression using lambda =  1e-05 is:  22.8850063597
MSE for linear regression using lambda =  0.0001 is:  22.8850072547
MSE for linear regression using lambda =  0.001 is:  22.8850162036
MSE for linear regression using lambda =  0.01 is:  22.8851055687
MSE for linear regression using lambda =  0.1 is:  22.8859868249
MSE for linear regression using lambda =  1.0 is:  22.8935623472
MSE for linear regression using lambda =  10.0 is:  22.8750069751
MSE for linear regression using lambda =  100.0 is:  22.7779605975
MSE for linear regression using lambda =  1000.0 is:  39.474646544
MSE for linear regression using lambda =  10000.0 is:  72.6952759129
MSE for linear regression using lambda =  1000000.0 is:  85.0336221078
The best lambda:  100
Ridge egression model trained on the entire set: 
MSE Score using best lambda =  100 :  15.5953601622


In [133]:
# LASSO (alpha hyperparameter)
print("Validation scores are as follows: ")
from sklearn import linear_model
alphaset = np.array([0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100])
for k in range(0,10):
    reg = linear_model.Lasso(alpha = alphaset[k])
    reg.fit(X_train_rr_lasso,y_train_rr_lasso)
    print("Score for alpha = ",alphaset[k], ': ', reg.score(X_val, y_val))
# need to compare this with y_val for different iterations of alpha. With the best alpha, need to train it again 
# on X_train and y_train and predict on X_test. 
print("The best alpha: ", 0.01)
print("Regression model trained on the entire set: ")
reg1 = linear_model.Lasso(alpha = 0.01)
reg1.fit(X_train,y_train)
print("Score using best alpha = ",0.01, ': ', reg1.score(X_test, y_test))
print("The coefficients are: ", reg1.coef_)
print("No. of non-zero elements in theta: ", len(reg1.coef_.nonzero()[0]))

Validation scores are as follows: 
Score for alpha =  1e-07 :  0.731298726628
Score for alpha =  1e-06 :  0.731298655974
Score for alpha =  1e-05 :  0.731297949379
Score for alpha =  0.0001 :  0.731290867182
Score for alpha =  0.001 :  0.731219033436
Score for alpha =  0.01 :  0.730397253034
Score for alpha =  0.1 :  0.723165756075
Score for alpha =  1.0 :  0.692256413639
Score for alpha =  10.0 :  -0.000414986887945
Score for alpha =  100.0 :  -0.000414986887945
The best alpha:  0.01
Regression model trained on the entire set: 
Score using best alpha =  0.01 :  0.661225050523
The coefficients are:  [-0.9875882   1.17407324  0.30151731  0.77439062 -1.85282595  2.60925289
 -0.         -2.93360802  2.77892596 -2.07389602 -2.07915671  0.9640536
 -4.27219691  0.        ]
No. of non-zero elements in theta:  12


In [134]:
y_pred = np.dot(X_test, reg1.coef_)
MSE = ((np.linalg.norm(y_test - y_pred))**2)/len(y_pred)
print("MSE for LASSO regression: ", MSE)

MSE for LASSO regression:  16.2357020162
